# Base Distillation

the baseline for the distillation matrics, including:
- a converged teacher ResNet-56 on CIFAR-100
- a converged student ResNet-20 on CIFAR-100
- a converged student ResNet-20 with Knowledge Distillation on CIFAR-100

the accuracy of these three model will served as the benchmark for the rest experient

In [2]:
from utils.download_cifar100 import download_cifar100

# 先加载数据集 CIFAR-100
trainset, testset, trainloader, testloader = download_cifar100()

开始下载CIFAR-100数据集...
下载训练集中...
下载测试集中...
下载完成！
训练集样本数: 50000
测试集样本数: 10000
类别数: 100


In [ ]:
from utils.base_model import (
    train_teacher_model,
    train_student_vanilla,
    train_student_with_distillation,
)
import torch


def run_all_benchmarks():
    """运行所有基准模型训练"""
    print("=" * 60)
    print("开始训练所有基准模型...")
    print("=" * 60)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用设备: {device}")

    results = {}

    # 1. 训练教师模型
    print("\n1. 训练教师模型 ResNet-56")
    teacher_model, teacher_losses, teacher_accuracies = train_teacher_model()
    results["teacher"] = {
        "model": teacher_model,
        "best_acc": max(teacher_accuracies) if teacher_accuracies else 0,
    }

    # 2. 训练学生模型（无蒸馏）
    print("\n2. 训练学生模型 ResNet-20（无蒸馏）")
    student_vanilla, vanilla_acc = train_student_vanilla()
    results["student_vanilla"] = {"model": student_vanilla, "best_acc": vanilla_acc}

    # 3. 训练学生模型（带蒸馏）
    print("\n3. 训练学生模型 ResNet-20（带知识蒸馏）")
    student_kd, kd_acc = train_student_with_distillation(teacher_model)
    results["student_kd"] = {"model": student_kd, "best_acc": kd_acc}

    # 打印最终结果比较
    print("\n" + "=" * 60)
    print("基准模型训练完成！结果汇总:")
    print("=" * 60)
    print(f"教师模型 (ResNet-56): {results['teacher']['best_acc']:.2f}%")
    print(f"学生模型无蒸馏 (ResNet-20): {results['student_vanilla']['best_acc']:.2f}%")
    print(f"学生模型带蒸馏 (ResNet-20): {results['student_kd']['best_acc']:.2f}%")

    # 计算提升
    improvement = (
        results["student_kd"]["best_acc"] - results["student_vanilla"]["best_acc"]
    )
    print(f"知识蒸馏带来的准确率提升: {improvement:+.2f}%")

    return results


def plot_comparison(results):
    """绘制结果比较图"""
    import matplotlib.pyplot as plt

    models = [
        "Teacher\nResNet-56",
        "Student\nNo KD\nResNet-20",
        "Student\nWith KD\nResNet-20",
    ]
    accuracies = [
        results["teacher"]["best_acc"],
        results["student_vanilla"]["best_acc"],
        results["student_kd"]["best_acc"],
    ]

    plt.figure(figsize=(10, 6))
    bars = plt.bar(models, accuracies, color=["#FF6B6B", "#4ECDC4", "#45B7D1"])
    plt.ylabel("Test Accuracy (%)")
    plt.title("Knowledge Distillation Benchmark Results on CIFAR-100")
    plt.ylim(0, 100)

    # 在柱状图上显示数值
    for bar, acc in zip(bars, accuracies):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 1,
            f"{acc:.2f}%",
            ha="center",
            va="bottom",
        )

    plt.tight_layout()
    plt.savefig("benchmark_results.png", dpi=150, bbox_inches="tight")
    plt.show()


# 运行所有基准训练
results = run_all_benchmarks()

# 绘制比较图
plot_comparison(results)

print("\n🎉 所有基准模型训练完成！")
print("📁 模型已保存为:")
print("   - teacher_resnet56_best.pth")
print("   - student_vanilla_resnet20_best.pth")
print("   - student_kd_resnet20_best.pth")
print("📊 结果图已保存为: benchmark_results.png")

开始训练所有基准模型...
使用设备: cuda

1. 训练教师模型 ResNet-56
🚀 开始训练教师模型 ResNet-56...


Epoch 1/200:  44%|████▎     | 171/391 [00:30<00:32,  6.72it/s, Loss=4.600, Acc=1.30%]